In [23]:
from azure.core.credentials import AzureKeyCredential
import os
%reload_ext dotenv
%dotenv Labfiles/05-custom-entity-recognition/Python/custom-entities/.env
ls_prediction_endpoint = os.getenv('AI_SERVICE_ENDPOINT')
ai_resource_key = os.getenv('AI_RESOURCE_KEY')
print(ls_prediction_endpoint)

https://ner-language-instance.cognitiveservices.azure.com/


In [24]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

In [25]:
credentials = AzureKeyCredential(ai_resource_key)
ai_client = TextAnalyticsClient(endpoint=ls_prediction_endpoint, credential=credentials)

In [26]:
project = "CustomNER"
model="ExtractAds"
deployment = "production"

# Extract files from ads folder to 'batchedDocuments

In [31]:
# Read each text file in the ads folder
batchedDocuments = []
ads_folder = 'ads'
files = os.listdir(ads_folder)
for file_name in files:
    # Read the file contents
    text = open(os.path.join(ads_folder, file_name), encoding='utf8').read()
    batchedDocuments.append(text)

# Prepare model and extract entities

In [32]:
operation = ai_client.begin_recognize_custom_entities(batchedDocuments, project_name=project, deployment_name=deployment)

In [33]:
document_results = operation.result()

In [34]:
for doc, custom_entities_results in zip(files, document_results):
    print(doc)
    if custom_entities_results.kind == "CustomEntityRecognition":
        for entity in custom_entities_results.entities:
            print("Entity: ", entity.text, "Category: ", entity.category, "Confidence Score: ", entity.confidence_score)
    elif custom_entities_results.is_error:
        print("Document error: ", custom_entities_results.error)

test1.txt
Entity:  Bluetooth earbuds Category:  IFS Confidence Score:  0.97
Entity:  $100. Category:  PC Confidence Score:  0.96
Entity:  Sacramento Category:  LCN Confidence Score:  0.91
test2.txt
Entity:  Dog harness Category:  IFS Confidence Score:  0.95
Entity:  $20 Category:  PC Confidence Score:  0.93
Entity:  Tucson Category:  LCN Confidence Score:  0.9
test3.txt
Entity:  Lego box Category:  IFS Confidence Score:  0.97
Entity:  havana Category:  LCN Confidence Score:  0.94


# Analysis of the results,
for test 1 and test 2, results were fine. However for test 3, the model failed to recognize the price entities. I simply wrote the numbers as 500 dollars and 25, instead of usual $500 and $25.

## Entities 
 - IFS: Item for Sale
 - PC: Price
 - LCN: Location
